In [8]:
import pymorphy2
import nltk
import re

import artm

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

### Удаляем заголовки из файла

In [1]:
file = open('./PScience/vw_PScience.txt', 'r')
f = file.read()
file.close()
lines = f.split('\n')

In [2]:
file = open("./PScience/vw_without_titles.txt", 'w')
for i, line in enumerate(lines):
    end = line.find('@title')
    if i < len(lines) - 1:
        file.write(line[:end - 2] + '\n')
    else:
        file.write(line[:end - 2])
    

### Делим на батчи преобразованный файл

In [3]:
path = './PScience/vw_PScience_1.txt'

In [4]:
path_title = "./PScience/vw_without_titles.txt"

In [5]:
Batches_without_titles =  './PScience/without_titles/'

In [6]:
BATCHES_PATH = './PScience/batches/'

In [9]:
batch_vectorizer = artm.BatchVectorizer(data_path=path_title, 
                                                 data_format='vowpal_wabbit',
                                                 target_folder=Batches_without_titles)

In [10]:
dict_title = artm.Dictionary()
dict_title.gather(data_path=Batches_without_titles)
MIN_DF = 3
dict_title.filter(min_df=MIN_DF)

artm.Dictionary(name=2b587032-06dc-4531-b0da-47b9b73c0925, num_entries=32420)

In [11]:
dict_title.save_text('dict_titles.txt')

## Создаем модель:

In [12]:
model = artm.ARTM(num_topics=20,
                  num_processors=7,
                  theta_columns_naming='title',
                  show_progress_bars=True,
                  class_ids={'word':10., '2gramm':1., "3gramm":1.,"author":1., "post_tag":100.})

model.initialize(dict_title)

In [13]:
model.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model.scores.add(artm.PerplexityScore(name='perplexity', class_ids={'word':10.0, '2gramm':1., "3gramm":1.,\
                             "author":1., "post_tag":100.},#, '@title':1., '@snippet':1.},
                                      dictionary=dict_title))
model.scores.add(artm.TopTokensScore(name='TopTokens_word', class_id='word', num_tokens=20))

In [14]:
model.num_document_passes = 10
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=50)

In [15]:
phi = model.get_phi()
theta = model.transform(batch_vectorizer=batch_vectorizer)

In [16]:
phi.to_csv('./PScience/phi.csv')
theta.to_csv('./PScience/theta.csv')

### Смотрим на получившиеся темы:

In [18]:
for topic in phi.keys():
    print('----------------\n')
    print(topic, '>>>')
    print(model.score_tracker['TopTokens_word'].last_tokens[topic])
    print('----------------\n\n')

----------------

topic_0 >>>
['мозг', 'материал', 'нейрон', 'pcourse', 'num', 'структура', 'память', 'молекула', 'химия', 'метод', 'образ', 'использовать', 'вещество', 'соединение', 'получить', 'область', 'являться', 'связь', 'помощь', 'процесс']
----------------


----------------

topic_1 >>>
['город', 'пространство', 'музей', 'место', 'городской', 'дом', 'центр', 'здание', 'москва', 'архитектура', 'большой', 'стать', 'территория', 'жизнь', 'россия', 'жить', 'регион', 'производство', 'новый', 'образ']
----------------


----------------

topic_2 >>>
['страна', 'экономика', 'китай', 'россия', 'цена', 'сша', 'компания', 'деньга', 'экономический', 'рынок', 'работа', 'большой', 'экономист', 'государство', 'стать', 'проблема', 'ситуация', 'кризис', 'решение', 'должный']
----------------


----------------

topic_3 >>>
['право', 'философия', 'говорить', 'закон', 'философ', 'свобода', 'должный', 'бог', 'являться', 'идея', 'философский', 'понятие', 'жизнь', 'миф', 'вопрос', 'представление',

## Птаемся формализовать понятие "монотематичность" и оценить количество "монотематических документов" для каждой темы
- Пока решила, что распределение тем в монотематическом документе такое: один максимум(topicMax) и значения для остальных тем $< a*topicMax$($a$ подбирается, сейчас около 0.1)

In [200]:
theta_val = theta.values
theta_copy = theta_val.copy()

(20, 3404)

In [20]:
mono_document_count = np.zeros(20)
mono_document_list = [[] for i in range(20)]

In [204]:
top_bound = 0.6
low_bound = 0.3
for i in range(theta_val.shape[1]):
    topic_argmax = np.argmax(theta_val[:, i])
    topic_max = np.max(theta_val[:, i])
    theta_copy[topic_argmax, i] = 0
    if topic_max > top_bound and np.sum(theta_copy[:, i] > bound*topic_max) == 0:
        mono_document_count[topic_argmax]+=1
        mono_document_list[topic_argmax].append([i, topic_max])
    

### Так выглядит счетчик количества монотематичеких документов:

In [205]:
mono_document_count

array([112.,  38.,  99.,  79.,  70., 153.,  35.,  92., 108.,  82.,  57.,
        47.,  89.,  67., 104.,  58., 136., 134.,  67., 160.])

###### Это список соответствующих документов, выберем 5 тем и 20 документов в каждом, попробуем построить на этом модель и проверить, как она работает. Для этого запомним, к каким темам относила эти документы "большая модель" и подсчитаем качество тематизирования, полагая полученны темы ответами

### Проблема:
- как понять, что темы не перемешались? 
  
- Пока руками выберем темы 2, 3, 6, 9,11. Как это автоматизировать?
- Получим индексы нужных нам документов
 

In [21]:
from random import shuffle
def choose_idx(topic_list, mono_list):
    collection_list = []
    for topic_id, n_doc in topic_list:
        doc_idxs = sorted(mono_list[topic_id], key = lambda item: item[1], reverse=True)[:n_doc]
        for doc in doc_idxs:
            collection_list.append([topic_id, doc[0]])
    shuffle(collection_list)
    return collection_list

test_95 = choose_idx([[5, 95], [8, 95], [16, 95], [17, 95], [19, 95]], mono_document_list)

#### Делаем новый файл .vw

In [172]:
def create_uniform_collection(num, doc_list):
    file = open("./PScience/vw_without_titles.txt", 'r')
    f = file.read()
    file.close()
    lines = f.split('\n')
    file = open("./PScience/vw_collections/uniform_" + str(num) + ".txt", "w")
    for i, doc_id in enumerate(doc_list):
        if i < len(doc_list) - 1:
            file.write(lines[doc_id[1]] + "\n")
        else:
            file.write(lines[doc_id[1]])
    file.close()
create_uniform_collection(95, test_95)

In [77]:
create_uniform_collection(20, test_ids)
create_uniform_collection(38, test_38)
create_uniform_collection(100, test_100)

'90'

#### Строим модель уже на новом файле

In [181]:
def create_uniform_model(num, num_topics):
    path = "./PScience/vw_collections/uniform_" + str(num) + ".txt"
    batch_folder = "./PScience/vw_collections/uniform_" + str(num) + "_batch"
    batch_vectorizer = artm.BatchVectorizer(data_path=path, 
                                                 data_format='vowpal_wabbit',
                                                 target_folder=batch_folder)
    uniform_dict  = artm.Dictionary()
    uniform_dict.gather(data_path=batch_folder)
    MIN_DF = 2
    MAX_DF = 70
    uniform_dict.filter(min_df=MIN_DF, max_df=MAX_DF)

    uniform = artm.ARTM(num_topics=num_topics,
                  num_processors=2,
                  theta_columns_naming='title',
                  show_progress_bars=True,
                  class_ids={'word':10., '2gramm':1., "3gramm":1.,"author":1., "post_tag":10., "class_id":1.})

    uniform.initialize(uniform_dict)

    uniform.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
    uniform.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
    uniform.scores.add(artm.PerplexityScore(name='perplexity', class_ids={'word':1.0, '2gramm':1., "3gramm":1.,\
                             "author":1., "post_tag":10.},#, '@title':1., '@snippet':1.},
                                      dictionary=uniform_dict))
    uniform.scores.add(artm.TopTokensScore(name='TopTokens_word', class_id='word', num_tokens=15))
    return uniform, batch_vectorizer

uniform_95,  batch_vectorizer = create_uniform_model(95, 5)

In [ ]:
uniform_20 = create_uniform_model(20, 5)
uniform_38 = create_uniform_model(38, 5)
uniform_100 = create_uniform_model(100, 5)

In [182]:
uniform_95.num_document_passes = 10
uniform_95.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=50)

In [183]:
phi95 = uniform_95.get_phi()

In [184]:
for topic in phi95.keys():
    print('----------------\n')
    print(topic, '>>>')
    print(uniform_95.score_tracker['TopTokens_word'].last_tokens[topic])
    #print(phi[topic].nlargest(20))
    print('----------------\n\n')

----------------

topic_0 >>>
['клетка', 'частица', 'днк', 'кварк', 'молекула', 'энергия', 'ген', 'рнк', 'нейтрино', 'организм', 'эксперимент', 'белка', 'бактерия', 'ускоритель', 'электрон']
----------------


----------------

topic_1 >>>
['звезда', 'галактика', 'солнце', 'симметрия', 'вселенная', 'планета', 'частица', 'диск', 'скорость', 'расстояние', 'вещество', 'свет', 'вращение', 'планет', 'наблюдение']
----------------


----------------

topic_2 >>>
['проект', 'сша', 'университет', 'экономика', 'компания', 'цена', 'правительство', 'прочитать', 'модель', 'инфляция', 'глава', 'анализ', 'штат', 'изобретение', 'политика']
----------------


----------------

topic_3 >>>
['русский', 'традиция', 'государство', 'власть', 'текст', 'бог', 'историк', 'исторический', 'народ', 'искусство', 'право', 'эпоха', 'глагол', 'политический', 'прошлое']
----------------


----------------

topic_4 >>>
['ребёнок', 'сталин', 'внимание', 'женщина', 'образование', 'социальный', 'предмет', 'москва', 'живо

Сопоставим каждому документу доминирующую тему

In [116]:
theta_un = uniform_90.transform(batch_vectorizer=batch_vectorizer)
theta_numpy = theta_un.values
theta_numpy.shape
topic = np.argmax(theta_numpy, axis=0)

In [117]:
count = np.array([0, 0, 0, 0])
for arg in topic:
    count[arg] +=1

In [111]:
true = 0
total = 0
for i in range(topic.shape[0]):
    if test_90[i][0] == topic[i]:
        true += 1
    total += 1
true/total

0.22562674094707522